In [1]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd
import numpy as np
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import time
import random
import re
from sportsreference.ncaab.boxscore import Boxscore
from sportsreference.ncaab.roster import Player
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [2]:
lst1 = []
lst2 = []
count = 0
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2012&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=0'
for i in range(45):
    if i >0:
        href_list = soup.find("div", {"class": "p402_premium"}).find('p').find_all('a')
        if len(href_list) == 1:
            count += 1
            print(count)
            if count == 2:
                break
            href = href_list[0].get('href')
        else:
            href = href_list[1].get('href')
        print(href)
        draft_url= 'https://www.basketball-reference.com' + href
        time.sleep(4*random.random())
    r = requests.get(draft_url)
    print(r.status_code)
    soup = BeautifulSoup(r.content, "html")
    div = soup.find("div", {"class": "table_outer_container"})
    table = div.find("tbody")
    rows = table.find_all('tr')
    for row in rows:
        tds = row.find_all('td')
        lst3 = []
        for item in tds:
            lst3.append(item.get_text())
        lst1.append(lst3)
for i in rows[0].find_all('td'):
    lst2.append(i.get("data-stat"))
        

200
1
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2012&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=100
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2012&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=200
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2012&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_

In [3]:
print(len(lst1))
#pd.DataFrame(lst1[1], index=lst2)
lst2

2098


['year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_country',
 'college',
 'year_min',
 'year_max',
 'g',
 'mp_per_g',
 'pts_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'fg_pct',
 'fg2_pct',
 'fg3_pct',
 'ft_pct',
 'ws',
 'ws_per_48']

In [4]:
all_star_list = []
url = 'https://www.basketball-reference.com/allstar/NBA-allstar-career-stats.html'
r = requests.get(url)
soup1 = BeautifulSoup(r.content, "html")
div = soup1.find("div", {"class": "table_outer_container"})
table = div.find("tbody")
rows = table.find_all('tr')
for row in rows:
    a_tag =row.find('th').find('a')
    if a_tag == None:
        continue
    player = a_tag.get_text()
    href = a_tag.get('href')
    all_star_list.append(player)
data = pd.DataFrame()

In [5]:
for playr in lst1:
    print('.', end='')
    try:
        player = playr[5]
    except:
        continue
    split_player = player.split()
    formatted_player = '-'.join(split_player)
    try:
        final_format = (formatted_player +'-1').lower().replace("'", "")
    except:
        continue
    #print(final_format)
    print(player)

    try:
        name = Player(final_format).dataframe.dropna(axis=1).iloc[-1,:]
    except:
        continue
    name['conference'] = Player(final_format).dataframe.dropna(axis=1).iloc[-2,:]['conference']
    name['years_in_college'] = len(Player(final_format).dataframe.dropna(axis=1).index) - 1
    name['nba_position'] = playr[7]
    data = pd.concat([data, pd.DataFrame(name.values.reshape(1,len(name.values)),
                                         index=[player],
                                         columns=[name.index])])
data

.Karl Malone
.LeBron James
.Tim Duncan
.Dirk Nowitzki
.Kevin Garnett
.Shaquille O'Neal
.David Robinson
.Reggie Miller
.Kobe Bryant
.Chris Paul
.Paul Pierce
.Gary Payton
.Ray Allen
.Pau Gasol
.Kevin Durant
.Jason Kidd
.Steve Nash
.Dwight Howard
.Patrick Ewing
.Vince Carter
...Scottie Pippen
.Shawn Marion
.James Harden
.Chauncey Billups
.Dwyane Wade
.Horace Grant
.Dikembe Mutombo
.Tony Parker
.Terry Porter
.Elton Brand
.Detlef Schrempf
.Jeff Hornacek
.LaMarcus Aldridge
.Manu Ginóbili
.Chris Bosh
.Rasheed Wallace
.Stephen Curry
.Jason Terry
.Tyson Chandler
.Carmelo Anthony
...Andre Miller
.Eddie Jones
.Grant Hill
.A.C. Green
.Allen Iverson
.Tracy McGrady
.Russell Westbrook
.Vlade Divac
.Andre Iguodala
.Chris Mullin
.Kevin Johnson
.Amar'e Stoudemire
.Mark Jackson
.Rashard Lewis
.Hersey Hawkins
.Dennis Rodman
.P.J. Brown
.Charles Oakley
.Clifford Robinson
.Alonzo Mourning
...Shawn Kemp
.Glen Rice
.Anthony Mason
.Paul Millsap
.Antawn Jamison
.Sam Cassell
.Dale Davis
.Kyle Lowry
.Joe Dumars
.

.Lionel Simmons
.T.J. Ford
.Kyle O'Quinn
.Lance Stephenson
.Jud Buechler
.Fred Jones
...Travis Outlaw
.Scott Burrell
.Jordan Hill
.Will Barton
.Sean Rooks
.Sasha Vujačić
.Craig Smith
.Daniel Gibson
.Mike Brown
.Darius Songaila
.Duane Causwell
.Brian Skinner
.Dan Gadzuric
.Jarron Collins
.Larry Krystkowiak
.Tony Massenburg
.Michael Beasley
.Vitaly Potapenko
.Jacque Vaughn
.Brian Cardinal
...Jared Sullinger
.Brad Lohaus
.Jumaine Jones
.Etan Thomas
.Tom Hammonds
.Luther Head
.Terrence Jones
.Tim Perry
.Avery Bradley
.Steve Novak
.Dino Rađa
.Winston Garland
.Samaki Walker
.Speedy Claxton
.Rudy Fernández
.Dean Garrett
.Jason Smith
.Derrick Williams
.Elliot Perry
.Jordan Farmar
.Chase Budinger
.Trenton Hassell
.Lavoy Allen
.Don Reid
.Walter McCarty
.Linas Kleiza
.Jason Caffey
.Luke Walton
.E'Twaun Moore
.Meyers Leonard
.Robert Traylor
.Chris Mihm
.Mike Scott
.Scott Padgett
.Jon Leuer
.Jared Jeffries
.Bryant Reeves
.Marc Jackson
.Roger Mason
.Tyrus Thomas
...Alec Burks
.Lawrence Funderburke
.

.Doug Edwards
.Dwayne Morton
.Jerome Allen
.Efthimi Rentzias
.Steve Hamer
.Marko Milič
.Kebu Stewart
.Roshown McLeod
.Ryan Stack
.Leon Smith
.Tim Young
.Erick Barkley
...Chris Porter
.Ruben Boumtje-Boumtje
.Travis Hansen
.Luke Jackson
.Dijon Thompson
.Marcus Vinicius
.Nathan Jawai
.Sasha Kaun
.Victor Claver
.Jermaine Taylor
.Lester Hudson
.Lazar Hayward
.Willie Warren
.JaJuan Johnson
.Jared Cunningham
.Kim English
.Ognjen Kuzmić
.Harold Keeling
.Cozell McQueen
.Scott Meents
...Conner Henry
.Curtis Kitchen
.Winston Crite
.Nate Johnston
.Gerald Paddio
.Gary Leonard
.Bo Kimble
.Les Jepsen
.Mark Macon
.Chad Gallagher
.Robert Werdann
.Geert Hammink
.Malcolm Mackey
.Evers Burns
.Gaylon Nickerson
.Martin Lewis
.Ben Davis
.Johnny Taylor
.Ed Gray
.Bubba Wells
...Eric Washington
.Korleone Young
.Tyson Wheeler
.Kaniel Dickens
.Igor Rakočević
.Antonis Fotsis
.Alvin Jones
.Frank Williams
.Corsley Edwards
.Ndudi Ebi
.James Lang
.Andre Emmett
.Antonio Burks
.Uroš Slokar
.Cedric Simmons
.Dee Brown
.Ja

.Dennis Williams
.Vernon Carr
.Carl Lott
.Brent Counts
.Michael Tait
.Anthony Wilson
.Mike Milling
.Andre LaFleur
.Terry Williams
.Patrick Fairs
.Frank Ross
.David Moss
.Brian Vaughns
.Gerry Wright
.Sam Hill
...Jose Antonio Montero
.Dave Butler
.Kenny Travis
.Howard Triche
.Perry Bromwell
.Ricky Brown
.Darryl Thomas
.Harold Jensen
.Marcel Boyce
.Kelvin Scarborough
.Tom Gneiting
.Doug Altenberger
.Gary Graham
.Dwayne Scholten
.Fred Jenkins
.Art Sabb
.Tracy Foster
.Bernard Johnson
.Gay Elmore
.Antoine Joubert
.Quintan Gates
.Riccardo Morandotti
.Tim Naegeli
.Frank Ford
.Henry Carr
.Frank Booker
.Wayne Williams
.Raynard Davis
.Scott Adubato
.Michael Foster
.Roy Singleton
.Rowan Gomes
.Mike Giomi
.Earvin Leavy
.Montel Hatcher
.Jamie Dixon
.Clarence Grier
.Ronnie Leggette
.Keith Webster
.Eric Semisch
...Kenny Stone
.Curtis Hunter
.Mark Gottfried
.Gerald White
.Franjo Arapovic
.Jerry Corcoran
.Ron Vanderschaaf
.Jeff Moe
.Jose Vargas
.Derrick Hamilton
.Jorge Gonzalez
.Rodney Johns
.Herman Mon

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,conference,defensive_box_plus_minus,defensive_rebound_percentage,defensive_rebounds,defensive_win_shares,...,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,weight,win_shares,win_shares_per_40_minutes,years_in_college
Karl Malone,NaN,125,NaN,59,NaN,southland,NaN,NaN,NaN,NaN,...,15.5,269,NaN,NaN,NaN,NaN,250,NaN,NaN,3
Tim Duncan,NaN,288,NaN,481,NaN,acc,NaN,NaN,NaN,NaN,...,16.2,335,1315,0.588,773,NaN,248,NaN,NaN,4
Shaquille O'Neal,NaN,152,NaN,412,NaN,sec,NaN,NaN,NaN,NaN,...,15.6,295,1289,0.61,786,NaN,325,NaN,NaN,3
David Robinson,NaN,82,NaN,516,NaN,colonial,NaN,NaN,NaN,NaN,...,10.9,261,NaN,NaN,NaN,NaN,235,NaN,NaN,4
Reggie Miller,NaN,247,NaN,22,NaN,pac-10,NaN,NaN,NaN,NaN,...,11.5,218,NaN,NaN,NaN,NaN,185,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stanley Robinson,NaN,131,NaN,130,NaN,big-east,NaN,NaN,511,7.4,...,15.6,210,796,0.52,414,NaN,210,11.8,0.143,4
Dwayne Collins,NaN,109,NaN,93,NaN,acc,NaN,NaN,506,7.1,...,16.9,222,820,0.565,463,NaN,241,13.2,0.177,4
Jon Diebler,NaN,269,NaN,24,NaN,big-ten,NaN,NaN,358,6.2,...,12,169,209,0.474,99,NaN,205,17.3,0.148,4
Ater Majok,1.9,4,10.1,42,NaN,big-east,NaN,12.3,47,0.6,...,23.2,21,59,0.424,25,11.6,233,0.5,0.051,1


In [6]:
count = 0
for player in lst1:
    count+=1
    try:
        if player[5] == 'Randy Holcomb':
            break
    except:
        continue

In [7]:
len(lst1)

2098

In [ ]:
#lst1[0][7]

In [9]:
nba = []
for player in data.index:
    if player in all_star_list:
        nba.append(1)
    else: 
        nba.append(0)

In [10]:
data.columns = [col[0] for col in data.columns]

In [11]:
#data.to_pickle('drafted_players_college.pkl')

In [21]:
data = pd.read_pickle('drafted_players_college.pkl')

In [22]:
def height_func(row):
    try: 
        row['height'].split('-')
    except:
        return 6.56
    split = row['height'].split('-')
    return int(split[0])+(int(split[1])/12)

In [23]:
data['height'] = data.apply(height_func ,axis=1)

In [24]:
data2 = data.copy()

In [25]:
data2.drop(columns=['player_id','team_abbreviation','position','nba_position','conference'],inplace = True)

In [26]:
for column in data2.columns:
    data2[column].astype(float)
    data2[column].fillna((data2[column].mean()), inplace=True) 

In [27]:
boost = GradientBoostingClassifier(learning_rate=.01, n_estimators=5000)

In [28]:
boost.fit(data2, nba)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=5000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [19]:
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=2013&year_max=2016&college_id=0&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&order_by=ws'
r = requests.get(draft_url)
soup = BeautifulSoup(r.content, "html")
draftees_after_2013 = []
count = 0
for i in range(3):
    if i >0:
        href_list = soup.find("div", {"class": "p402_premium"}).find('p').find_all('a')
        if len(href_list) == 1:
            count +=1
            print(count)
            if count == 2:
                break
            href = href_list[0].get('href')
        else:
            href = href_list[1].get('href')
        print(href)
        draft_url= 'https://www.basketball-reference.com' + href
        time.sleep(4*random.random())
    r = requests.get(draft_url)
    print(r.status_code)
    soup = BeautifulSoup(r.content, "html")
    div = soup.find("div", {"class": "table_outer_container"})
    table = div.find("tbody")
    rows = table.find_all('tr')
    for row in rows:
        tds = row.find_all('td')
        lst3 = []
        for item in tds:
            lst3.append(item.get_text())
        draftees_after_2013.append(lst3)

200
1
/play-index/draft_finder.cgi?request=1&year_min=2013&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=100
200
/play-index/draft_finder.cgi?request=1&year_min=2013&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=200
200


In [120]:
data_after_2013 = pd.DataFrame()
count = 0
data
print(len(draftees_after_2013))
for playr in draftees_after_2013:
    print('.', end='')
    try:
        player = playr[5]
    except:
        continue
    split_player = player.split()
    formatted_player = '-'.join(split_player)
    try:
        final_format = (formatted_player +'-1').lower().replace("'", "")
    except:
        continue
    if player == 'Larry Nance':
        final_format ='larry-nance-2'
    if player == 'Tim Hardaway':
        final_format = 'tim-hardaway-jr-1'
    if player == 'Glenn Robinson':
        final_format = 'glenn-robinson-2'
    if player == 'Glen Rice':
        final_format = 'glen-rice-jr-1'
    #print(final_format)
    print(player)

    try:
        name = Player(final_format).dataframe.dropna(axis=1).iloc[-1,:]
    except:
        continue
    name['conference'] = Player(final_format).dataframe.dropna(axis=1).iloc[-2,:]['conference']
    name['years_in_college'] = len(Player(final_format).dataframe.dropna(axis=1).index) - 1
    name['nba_position'] = playr[7]
    data_after_2013 = pd.concat([data_after_2013, pd.DataFrame(name.values.reshape(1,len(name.values)),
                                         index=[player],
                                         columns=[name.index])])
    count += 1
    print(count)
data_after_2013

258
.Giannis Antetokounmpo
.Rudy Gobert
.Karl-Anthony Towns
1
.Nikola Jokić
.Steven Adams
2
.Clint Capela
.Mason Plumlee
3
.Otto Porter
4
.CJ McCollum
5
.Kelly Olynyk
6
.Gorgui Dieng
7
.Victor Oladipo
8
.Cody Zeller
9
.Kentavious Caldwell-Pope
10
.Dwight Powell
11
.Myles Turner
12
.Montrezl Harrell
13
.Jerami Grant
14
.Aaron Gordon
15
.Julius Randle
16
...Ben Simmons
17
.Willie Cauley-Stein
18
.Joel Embiid
19
.Nerlens Noel
20
.Jusuf Nurkić
.Larry Nance
21
.Marcus Smart
22
.Gary Harris
23
.Rodney Hood
24
.Josh Richardson
25
.Alex Len
26
.Pascal Siakam
27
.Kyle Anderson
.Tim Hardaway
28
.Allen Crabbe
29
.Tony Snell
30
.Spencer Dinwiddie
31
.Malcolm Brogdon
32
.Elfrid Payton
33
.Dennis Schröder
...Domantas Sabonis
34
.Andrew Wiggins
35
.T.J. Warren
.Andre Roberson
36
.Richaun Holmes
37
.Jakob Pöltl
.Kristaps Porziņģis
.Jamal Murray
38
.Mike Muscala
39
.James Ennis
40
.Jabari Parker
41
.Dario Šarić
.Frank Kaminsky
42
.Doug McDermott
43
.Jordan Clarkson
44
.Delon Wright
45
.Joe Harris
46
.K

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,conference,defensive_box_plus_minus,defensive_rebound_percentage,defensive_rebounds,defensive_win_shares,...,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,weight,win_shares,win_shares_per_40_minutes,years_in_college
Karl-Anthony Towns,11.6,44,11.5,88,17.3,sec,10.8,22.3,169,2.9,...,14.2,53,248,0.577,143,23.7,250,6.4,0.311,1
Steven Adams,5.4,20,11.2,65,11.9,big-east,8.8,21,113,1.9,...,14.4,35,175,0.571,100,17.7,250,3.5,0.185,1
Mason Plumlee,11.1,212,5,199,NaN,acc,NaN,23.7,755,8.2,...,19.2,282,905,0.578,523,20.2,235,16.6,0.181,4
Otto Porter,14.9,135,3.1,55,11.4,big-east,5.6,20.3,336,5.3,...,10.8,85,433,0.55,238,20.9,205,12,0.231,2
CJ McCollum,20.1,304,1.5,51,NaN,patriot,NaN,17,529,6.6,...,11.3,267,1170,0.47,550,32.4,180,20.5,0.224,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aaron White,NaN,173,NaN,4,NaN,independent,NaN,NaN,155,2.1,...,18.9,169,245,0.42,103,NaN,185,5.2,0.103,4
Tyler Harvey,15.9,197,0.6,16,2.6,big-sky,-3.8,10.5,244,1.5,...,10.8,151,464,0.504,234,26.7,185,12.3,0.187,3
Sir'Dominic Pointer,16.1,299,5,173,6.6,big-east,5.5,16.1,520,6.4,...,17.6,218,743,0.518,385,17.3,192,10.9,0.119,4
Cady Lalanne,1.9,26,6.8,197,2.4,atlantic-10,2.4,21.5,540,4.7,...,17.9,206,720,0.575,414,19.9,253,9.7,0.139,4


In [121]:
data_after_2013.columns = [col[0] for col in data_after_2013.columns]

In [122]:
data_after_2013.to_pickle('after_2013_drafted_players_college.pkl')

In [123]:
'Tim Hardaway' in data_after_2013.index

True

In [124]:
#data_after_2013 = pd.read_pickle('after_2013_drafted_players_college.pkl')

In [125]:
nba2 = []
for player in data_after_2013.index:
    if player in all_star_list:
        nba2.append(1)
    else: 
        nba2.append(0)

In [126]:
data_after_2013['height'] = data_after_2013.apply(height_func ,axis=1)

In [127]:
data_after_2013_1 = data_after_2013.copy()

In [128]:
data_after_2013_1.columns

Index(['assist_percentage', 'assists', 'block_percentage', 'blocks',
       'box_plus_minus', 'conference', 'defensive_box_plus_minus',
       'defensive_rebound_percentage', 'defensive_rebounds',
       'defensive_win_shares', 'effective_field_goal_percentage',
       'field_goal_attempts', 'field_goal_percentage', 'field_goals',
       'free_throw_attempt_rate', 'free_throw_attempts',
       'free_throw_percentage', 'free_throws', 'games_played', 'games_started',
       'height', 'minutes_played', 'nba_position', 'offensive_box_plus_minus',
       'offensive_rebound_percentage', 'offensive_rebounds',
       'offensive_win_shares', 'personal_fouls', 'player_efficiency_rating',
       'player_id', 'points', 'points_produced', 'position',
       'steal_percentage', 'steals', 'team_abbreviation',
       'three_point_attempt_rate', 'three_point_attempts',
       'three_point_percentage', 'three_pointers', 'total_rebound_percentage',
       'total_rebounds', 'true_shooting_percentage', 'tu

In [131]:
data_after_2013_1.drop(columns=['player_id','conference','team_abbreviation','position','nba_position'],inplace = True)

KeyError: "['player_id' 'conference' 'team_abbreviation' 'position'] not found in axis"

In [132]:
data_after_2013_1.drop(columns=['nba_position'],inplace = True)

In [133]:
for column in data_after_2013_1.columns:
    print(column)
    data_after_2013_1[column].astype(float)
    data_after_2013_1[column].fillna((data_after_2013_1[column].mean()), inplace=True) 

assist_percentage
assists
block_percentage
blocks
box_plus_minus
defensive_box_plus_minus
defensive_rebound_percentage
defensive_rebounds
defensive_win_shares
effective_field_goal_percentage
field_goal_attempts
field_goal_percentage
field_goals
free_throw_attempt_rate
free_throw_attempts
free_throw_percentage
free_throws
games_played
games_started
height
minutes_played
offensive_box_plus_minus
offensive_rebound_percentage
offensive_rebounds
offensive_win_shares
personal_fouls
player_efficiency_rating
points
points_produced
steal_percentage
steals
three_point_attempt_rate
three_point_attempts
three_point_percentage
three_pointers
total_rebound_percentage
total_rebounds
true_shooting_percentage
turnover_percentage
turnovers
two_point_attempts
two_point_percentage
two_pointers
usage_percentage
weight
win_shares
win_shares_per_40_minutes
years_in_college


In [134]:
boost.predict(data_after_2013_1)

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0])

In [136]:
print(len(nba2 != boost.predict(data_after_2013_1)))
print(212-len(data_after_2013_1.index[nba2 != boost.predict(data_after_2013_1)]))
data_after_2013_1.index[nba2 != boost.predict(data_after_2013_1)]

165
186


Index(['Steven Adams', 'Victor Oladipo', 'Cody Zeller', 'Ben Simmons',
       'Joel Embiid', 'Larry Nance', 'Marcus Smart', 'Tim Hardaway',
       'Elfrid Payton', 'Domantas Sabonis', 'Frank Kaminsky', 'Doug McDermott',
       'Delon Wright', 'Tyus Jones', 'Jahlil Okafor', 'Glenn Robinson',
       'Cameron Payne', 'Nate Wolters', 'Jarnell Stokes', 'Dakari Johnson',
       'Jordan Adams', 'Joel Bolomboy', 'Brice Johnson', 'Wade Baldwin',
       'Arsalan Kazemi', 'Cady Lalanne'],
      dtype='object')

In [137]:
yhat = boost.predict(data_after_2013_1)

In [138]:
tn, fp, fn, tp = confusion_matrix(nba2,boost.predict(data_after_2013_1)).ravel()

In [139]:
print((tn, fp, fn, tp))
data_after_2013_1.index[((np.array(yhat)==1) & (np.array(nba2)==1))]

(136, 20, 6, 3)


Index(['Karl-Anthony Towns', 'D'Angelo Russell', 'Glen Rice'], dtype='object')

In [ ]:
roc_auc_score(nba2,yhat)

In [ ]:
data_after_2013_2 = data_after_2013.copy()

In [ ]:
data_after_2013_2.drop(columns=['player_id'],inplace = True)

In [ ]:
data_after_2013_2 = pd.get_dummies(data=data_after_2013_2,columns=['position','team_abbreviation','nba_position','conference']) 

In [ ]:
for column in data_after_2013_2.columns:
    data_after_2013_2[column].astype(float)
    length_of_nulls=len(data_after_2013_2[data_after_2013_2[column].isnull()])
    if length_of_nulls > 0:
        data_after_2013_2[column +' is missing'] = data_after_2013_2[column].isnull()*1
        data_after_2013_2[column].fillna(-1, inplace=True) 
        

In [ ]:
boost2 = GradientBoostingClassifier(learning_rate=.01, n_estimators=5000)

In [ ]:
data3 = data.copy

In [ ]:
data3.drop(columns=['player_id'],inplace = True)

In [ ]:
data3 = pd.get_dummies(data=data3,columns=['position','team_abbreviation','nba_position','conference']) 

In [ ]:
for column in data_after_2013_2.columns:
    data_after_2013_2[column].astype(float)
    length_of_nulls=len(data_after_2013_2[data_after_2013_2[column].isnull()])
    if length_of_nulls > 0:
        data_after_2013_2[column +' is missing'] = data_after_2013_2[column].isnull()*1
        data_after_2013_2[column].fillna(-1, inplace=True)